# Sendy Logistics Challenge

### Problem Statement:
#### To predict the estimated time of arrival (ETA) for motorbike deliveries in Nairobi, Kenya.
(To predict the estimated time of delivery of orders, from the point of driver pickup to the point of arrival at final destination.)

https://zindi.africa/competitions/sendy-logistics-challenge

Logistics in Sub-Saharan Africa increases the cost of manufactured goods by up to 320%; while in Europe, it only accounts for up to 90% of the manufacturing cost. Economies are better when logistics is efficient and affordable and hence the challenge. The data consists of 29 attribute which shall help determine the time that is needed for delivery of a parcel made under Sendy, a delivery servive.

The solution will help enhance customer communication and improve the reliability of service; which will ultimately improve customer experience. In addition, the solution will enable Sendy to realize cost savings, and ultimately reduce the cost of doing business, through improved resource management and planning for order scheduling.

#### Team: D_06
#### Members:
266 Vibha Hegde

257 Tejaswini Kale

262 Usman Khan

273 K L Vijeth

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MinMaxScaler,normalize
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn import svm
from sklearn.metrics import r2_score

import xgboost as xgb
from copy import deepcopy
from datetime import datetime
from IPython.display import HTML
import json
%pylab inline



Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning:

pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/sendy")

# **Training dataset**

#### Reading Training data into a Pandas Data Frame.

In [0]:
col_names=['Order_No', 'User_Id', 'Vehicle_Type', 'Platform_Type',
       'Type', 'Placement_Day_of_Month',
       'Placement_Weekday', 'Placement_Time',
       'Confirmation_Day_of_Month', 'Confirmation_Weekday',
       'Confirmation_Time', 'Arrival_at_Pickup_Day_of_Month',
       'Arrival_at_Pickup_Weekday', 'Arrival_at_Pickup_Time',
       'Pickup_Day_of_Month', 'Pickup_Weekday', 'Pickup_Time',
       'Arrival_at_Destination_Day_of_Month',
       'Arrival_at_Destination_Weekday',
       'Arrival_at_Destination_Time', 'Distance', 'Temperature',
       'Precipitation', 'Pickup_Lat', 'Pickup_Long',
       'Destination_Lat', 'Destination_Long', 'Rider_Id',
       'Time_from_Pickup_to_Arrival']
       
data=pd.read_csv("Train.csv",names=col_names,skiprows=1)
data.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Type,Placement_Day_of_Month,Placement_Weekday,Placement_Time,Confirmation_Day_of_Month,Confirmation_Weekday,Confirmation_Time,Arrival_at_Pickup_Day_of_Month,Arrival_at_Pickup_Weekday,Arrival_at_Pickup_Time,Pickup_Day_of_Month,Pickup_Weekday,Pickup_Time,Arrival_at_Destination_Day_of_Month,Arrival_at_Destination_Weekday,Arrival_at_Destination_Time,Distance,Temperature,Precipitation,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Rider_Id,Time_from_Pickup_to_Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


#### Reading Rider information into a data frame.

In [0]:
ridCol=['Rider_Id','No_Of_Orders','Age','Average_Rating','No_of_Ratings']

In [0]:
riders=pd.read_csv('Riders.csv',names=ridCol,skiprows=1)
riders.head()

,Rider_Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176
2,Rider_Id_648,1746,821,14.3,466
3,Rider_Id_753,314,980,12.5,75
4,Rider_Id_335,536,1113,13.7,156


# Data Exploration

#### Analysis of Null values in training dataset.

In [0]:
data_nan=(data.isnull().sum() / data.shape[0]) * 100
data_nan[data_nan>0]

Temperature      20.593368
Precipitation    97.396349
dtype: float64

#### Number of orders placed on the four different platforms by platform type.

In [0]:
data.Platform_Type.value_counts()

3    18054
1     2147
2      980
4       20
Name: Platform_Type, dtype: int64

#### Conclusion on number of values in attributes Vehicle Type and Platform Type.

In [0]:
print(set(data['Vehicle_Type']))
print(set(data['Platform_Type']))

{'Bike'}
{1, 2, 3, 4}


 Vehicle Type Attribute has only one value - Bike. 

There are only 4 platform types that are numbered and not disclosed.

#### Histogram visualising the number of bookings done by each platform.

In [0]:
P1 = [[1,2147],[2,980],[3,18054],[4,20]]
hist=pd.DataFrame(P1,columns=['Platform_Type','Count'])
print(hist)
fig=px.bar(hist,x='Platform_Type',y='Count',color='Count',height=500)
fig.show()

   Platform_Type  Count
0              1   2147
1              2    980
2              3  18054
3              4     20


Note that the number of bookings from the 4th platform type are barely any and hence the bar is not visible. maximum orders are done using platform 3.

#### Histogram showing number of bookings by order type.

In [0]:
P1 = [['Business',17384],['Personal',3817]]
hist=pd.DataFrame(P1,columns=['Order_type','Count'])
print(hist)
fig=px.bar(hist,x='Order_type',y='Count',color='Count',height=500)
fig.show()

  Order_type  Count
0   Business  17384
1   Personal   3817


Most of the orders are business type. There is a chance of having bulky orders.

#### Analysis of Temperature attribute

In [0]:
box_data = [
    go.Box(
        y=data['Temperature'],
        boxpoints='all',
        jitter=0.3,
        pointpos=-1.8,
        name="Temperature Distribution"
    )
]
py.iplot(box_data)

The distribution of temperature is fairly constant.

#### Analysis of Distance between pick up and drop location.

In [0]:
box_data2 = [
    go.Box(
        y=data['Distance'],
        boxpoints='suspectedoutliers',
        jitter=0.3,
        pointpos=-1.8,
        name="Distance"
    )
]
py.iplot(box_data2)

The distance varies largely from a minimum of 1KM to a maximum of 49KM.

#### The Placement Day of Month Attribute

In [0]:
x=data['Placement_Day_of_Month'].value_counts(sort=True)
x=x.to_frame()
x['Day_of_month']=x.index
x['count']=x['Placement_Day_of_Month']
x.drop(['Placement_Day_of_Month'],axis=1,inplace=True)
fig2=px.bar(x,x='Day_of_month',y='count',color='count',height=10000)
fig2.show()

With this attribute we may notice some kind of seasonality in the data and lean towards time series analysis in further processing steps. 

#### Percieving time taken for delivery of a package and the distance it travelled in one variable - Speed.

In [0]:
data['speed']=data['Distance']*3600/data['Time_from_Pickup_to_Arrival']
line_data = data[['Distance','Time_from_Pickup_to_Arrival','speed']]
line_data.sort_values("speed",inplace = True)
line_data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Distance,Time_from_Pickup_to_Arrival,speed
11087,1,3912,0.920245
16384,2,7714,0.933368
17758,1,3394,1.060695
381,1,2997,1.201201
13020,1,2836,1.269394


#### Outliers in Speed implying noise in distance or time duration attribute.

In [0]:
data1 = line_data[line_data['speed']>80]
data1.shape[0]

966

In [0]:
data1['speed'].describe()

count      966.000000
mean      4984.404015
std       7498.835461
min         80.446927
25%        331.609195
50%       1800.000000
75%       7200.000000
max      61200.000000
Name: speed, dtype: float64

Common knowledge has it that no motorbike can have an average speed greater than say, 70 kmph. Giving a leeway of 10 kmph, we understand that the data has 966 outliers ranging from 81 Kmph to 65000Kmph which is humanly impossible. 

In [0]:
box_data3 = [
    go.Box(
        y=data1['speed'],
        boxpoints='all',
        jitter=0.3,
        pointpos=-1.8,
        name="Time"
    )
]
py.iplot(box_data3)

It is clear from this boxplot that these values need to be filtered before feeding into the model to get a near accurate result.

#### Summary:

From the exploratory data analysis, it is clear that the attributes that we need to choose carefully to build the model are the time related attributes of order placements, confirmation, pickup and estimate the delivery time. 

Factors like precipitation, even though very less in numberof values may play an important role especially considering the fact that the data belongs to the low precipitation areas of Kenya. 

Temperature may or may not affect the delivery. We need to look into how the platform on which the order has been placed affects the confirmation time and intern how it speeds up or delays the delivery process. It may not lead on to huge differences, it may help make the model more accurate.

Orders placed are high on certain days of month and week. Hence seasonality can be found. Time series analysis must be made. 

# **Data Pre-processing**

## Data Cleaning

### Missing Values
Temperature attribute is filled with mean.

Precipitation is filled with Zero.

In [0]:
mean_temp=data.Temperature.mean()
data['Temperature'].fillna(mean_temp,inplace=True)
data['Precipitation'].fillna(0,inplace=True)

## Data Integration
Training dataset is merged with relevent attributes of rider's dataset.


In [0]:
riders["Rider_exp"]=(riders['No_Of_Orders']/(riders["Age"]))*riders['Average_Rating']
riders.Rider_exp
data = pd.merge(data, riders[['Rider_Id','Rider_exp']], on='Rider_Id')
data.transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,21161,21162,21163,21164,21165,21166,21167,21168,21169,21170,21171,21172,21173,21174,21175,21176,21177,21178,21179,21180,21181,21182,21183,21184,21185,21186,21187,21188,21189,21190,21191,21192,21193,21194,21195,21196,21197,21198,21199,21200
Order_No,Order_No_4211,Order_No_27440,Order_No_14170,Order_No_21727,Order_No_8301,Order_No_10440,Order_No_25321,Order_No_2476,Order_No_14704,Order_No_118,Order_No_10887,Order_No_918,Order_No_15419,Order_No_267,Order_No_23527,Order_No_303,Order_No_6056,Order_No_26295,Order_No_26873,Order_No_22150,Order_No_28145,Order_No_7684,Order_No_5724,Order_No_25375,Order_No_14810,Order_No_16835,Order_No_1672,Order_No_12021,Order_No_21063,Order_No_25751,Order_No_15423,Order_No_10780,Order_No_9901,Order_No_17664,Order_No_2765,Order_No_8841,Order_No_3266,Order_No_13420,Order_No_15216,Order_No_3854,...,Order_No_5741,Order_No_1921,Order_No_3409,Order_No_21249,Order_No_23677,Order_No_13217,Order_No_1489,Order_No_297,Order_No_27402,Order_No_22888,Order_No_19971,Order_No_7005,Order_No_4843,Order_No_23091,Order_No_6421,Order_No_2078,Order_No_26796,Order_No_25524,Order_No_18705,Order_No_27093,Order_No_23810,Order_No_27812,Order_No_18086,Order_No_24548,Order_No_12301,Order_No_19006,Order_No_6098,Order_No_6655,Order_No_18827,Order_No_18182,Order_No_7344,Order_No_13073,Order_No_14073,Order_No_15727,Order_No_7218,Order_No_27103,Order_No_3786,Order_No_7281,Order_No_15575,Order_No_15578
User_Id,User_Id_633,User_Id_2642,User_Id_3339,User_Id_3523,User_Id_87,User_Id_2801,User_Id_1882,User_Id_2260,User_Id_2186,User_Id_3688,User_Id_2209,User_Id_1363,User_Id_53,User_Id_641,User_Id_3326,User_Id_2468,User_Id_3689,User_Id_2789,User_Id_1523,User_Id_393,User_Id_2801,User_Id_3175,User_Id_3771,User_Id_2285,User_Id_3698,User_Id_3291,User_Id_2639,User_Id_3547,User_Id_1004,User_Id_2066,User_Id_3698,User_Id_393,User_Id_1249,User_Id_3525,User_Id_265,User_Id_2330,User_Id_1249,User_Id_2102,User_Id_896,User_Id_3222,...,User_Id_3291,User_Id_1151,User_Id_3283,User_Id_640,User_Id_3186,User_Id_3177,User_Id_2924,User_Id_2335,User_Id_1404,User_Id_1640,User_Id_2106,User_Id_2358,User_Id_2069,User_Id_3147,User_Id_1787,User_Id_2373,User_Id_3355,User_Id_2781,User_Id_2200,User_Id_173,User_Id_517,User_Id_265,User_Id_928,User_Id_2527,User_Id_3291,User_Id_1329,User_Id_2330,User_Id_2195,User_Id_46,User_Id_1329,User_Id_868,User_Id_2030,User_Id_136,User_Id_3599,User_Id_435,User_Id_3718,User_Id_1558,User_Id_2008,User_Id_1346,User_Id_2174
Vehicle_Type,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,...,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike,Bike
Platform_Type,3,3,3,3,2,3,1,1,1,3,3,3,3,3,3,3,3,3,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,...,3,2,3,3,3,3,2,3,2,2,1,4,3,3,3,1,3,3,3,3,3,3,3,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
Type,Business,Personal,Business,Personal,Personal,Business,Business,Personal,Personal,Business,Business,Business,Business,Personal,Business,Business,Business,Business,Personal,Business,Business,Business,Personal,Personal,Business,Business,Personal,Business,Business,Business,Business,Business,Business,Business,Business,Business,Business,Business,Personal,Business,...,Business,Personal,Business,Business,Business,Business,Personal,Business,Personal,Personal,Personal,Business,Business,Business,Business,Personal,Business,Personal,Business,Business,Business,Business,Business,Personal,Business,Business,Business,Business,Business,Business,Business,Business,Business,Business,Business,Business,Personal,Business,Business,Business
Placement_Day_of_Month,9,18,31,2,22,29,2,19,28,14,16,28,8,2,14,4,9,24,2,25,14,15,17,12,22,15,14,10,4,20,21,14,15

## Data transformation
One hot encoding of platform type ( 1, 2, 3, 4 ) and order type (Business,Personal)

In [0]:
int_data=LabelEncoder().fit_transform(data['Platform_Type'])
one_hot_data=OneHotEncoder().fit_transform(int_data.reshape(-1,1)).toarray()
data['1']=one_hot_data[:,0]
data['2']=one_hot_data[:,1]
data['3']=one_hot_data[:,2]
data['4']=one_hot_data[:,3]
#onehotencoder = OneHotEncoder(categorical_features = ['Platform_Type'])
#data = onehotencoder.fit_transform(data['Platform_Type']).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.



In [0]:
int_data=LabelEncoder().fit_transform(data['Type'])
one_hot_data=OneHotEncoder().fit_transform(int_data.reshape(-1,1)).toarray()
data['Business']=one_hot_data[:,0]
data['Personal']=one_hot_data[:,1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.



## Data reduction

### Attribute subset selection
1. Drop Order_No attribute as it is not contributing in prediction.

2. Drop Placement_Day_of_Month , Placement_Weekday,   
     Confirmation_Day_of_Month , Confirmation_Weekday,                                       
     Arrival_at_Pickup_Day_of_Month , Arrival_at_Pickup_Weekday ,                                                                                                          
     Pickup_Day_of_Month , Pickup_Weekday attributes                                
     beacuse they are done on the same day hence redundant values.


### Atrribute / Feature creation

Estimation of time elapsed using time atrributes

Time taken for confirmation

Time taken to arrive at pickup

Time taken to pickup


In [0]:
#difference in placement and confirmation times
def hfmt(x):
  return x.strftime('%H:%M:%S')
data['Placement_Time']=pd.to_datetime(data['Placement_Time'])
data['Placement_Time']=data['Placement_Time'].apply(hfmt)
data['Placement_Time']=pd.to_timedelta(data['Placement_Time'])

data['Confirmation_Time']=pd.to_datetime(data['Confirmation_Time'])
data['Confirmation_Time']=data['Confirmation_Time'].apply(hfmt)
data['Confirmation_Time']=pd.to_timedelta(data['Confirmation_Time'])

data['Arrival_at_Pickup_Time']=pd.to_datetime(data['Arrival_at_Pickup_Time'])
data['Arrival_at_Pickup_Time']=data['Arrival_at_Pickup_Time'].apply(hfmt)
data['Arrival_at_Pickup_Time']=pd.to_timedelta(data['Arrival_at_Pickup_Time'])

data['Pickup_Time']=pd.to_datetime(data['Pickup_Time'])
data['Pickup_Time']=data['Pickup_Time'].apply(hfmt)
data['Pickup_Time']=pd.to_timedelta(data['Pickup_Time'])
data['pickup_hour']=pd.to_datetime(data['Pickup_Time']).dt.hour

data['Arrival_at_Destination_Time']=pd.to_datetime(data['Arrival_at_Destination_Time'])
data['Arrival_at_Destination_Time']=data['Arrival_at_Destination_Time'].apply(hfmt)
data['Arrival_at_Destination_Time']=pd.to_timedelta(data['Arrival_at_Destination_Time'])
data['arrival_hour']=pd.to_datetime(data['Arrival_at_Destination_Time']).dt.hour
#time=pd.to_timedelta('00:01:00')
#data[data['Confirmation_Time']-data['Placement_Time']<time]['Placement_Weekday'].value_counts()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version



In [0]:
data['conf_no']=data['Confirmation_Time']-data['Placement_Time']
conf_temp=pd.DatetimeIndex(data['conf_no'])
conf_temp=(conf_temp.hour*60+conf_temp.minute+(conf_temp.second/60))
data['confirmation_time_min']=conf_temp

data['Arrival_no']=data['Arrival_at_Pickup_Time']-data['Confirmation_Time']
conf_temp=pd.DatetimeIndex(data['Arrival_no'])
conf_temp=(conf_temp.hour*60+conf_temp.minute+(conf_temp.second/60))
data['Arrival_time_min']=conf_temp

data['Pickup_no']=data['Pickup_Time']-data['Arrival_at_Pickup_Time']
conf_temp=pd.DatetimeIndex(data['Pickup_no'])
conf_temp=(conf_temp.hour*60+conf_temp.minute+(conf_temp.second/60))
data['Pickup_time_min']=conf_temp

data['Dest_no']=data['Arrival_at_Destination_Time']-data['Pickup_Time']
conf_temp=pd.DatetimeIndex(data['Dest_no'])
conf_temp=(conf_temp.hour*60+conf_temp.minute+(conf_temp.second/60))
data['Dest_time_min']=conf_temp

conf_temp=pd.DatetimeIndex(data['Pickup_Time'])
data['pick_hour']=conf_temp.hour

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version



Avarage speed of a rider and his experience.

In [0]:
data['speed']=data['Distance']*3600/data['Time_from_Pickup_to_Arrival']
line_data = data[['Distance','Time_from_Pickup_to_Arrival','speed']]
line_data.sort_values("speed",inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [0]:
rider_speeds=data.groupby(['Rider_Id'])
xx=rider_speeds['speed'].mean()/10
data.replace(xx.index,xx.values,inplace=True)

In [0]:
data['Rider_Id']=data['Rider_Id'].apply(pd.to_numeric)

## Model Building

Feature Selection

In [0]:
X=data[['Personal','Business','Pickup_time_min','Distance','Rider_Id','Rider_exp','Destination_Lat','Destination_Long','Pickup_Lat','Pickup_Long']].values
y=data['Dest_time_min'].values 

Split dataset in the ratio of 7:3 for testing.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

Feature Generation

In [0]:
polynomial_features= PolynomialFeatures(degree=2)
x_poly = polynomial_features.fit_transform(X_train)
x_test = polynomial_features.fit_transform(X_test)

Linear regression

In [0]:
Lin_reg=LinearRegression()

In [0]:
Lin_fit=Lin_reg.fit(x_poly,y_train)

In [0]:
Lin_pred = Lin_fit.predict(x_test)

In [62]:
test_df = pd.DataFrame({'Actual': y_test, 'Predicted': Lin_pred})
test_df.head()

,Actual,Predicted
0,8.250000,18.177476
1,22.366667,31.488975
2,68.133333,46.332550
3,32.033333,17.162891
4,10.466667,10.868591


In [63]:
metrics.mean_squared_error(y_test, test_df['Predicted'])

163.5835098490185

**Adaboost regresser**

In [ ]:
gsc = GridSearchCV(
        estimator=AdaBoostRegressor(base_estimator=xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 3, alpha = 10, n_estimators = 400)),
        param_grid={
            'learning_rate': (0.1,0.07,0.08),
            'n_estimators': (6,7,8),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,n_jobs=-1)
    
grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_

In [0]:
Ada_reg = AdaBoostRegressor(base_estimator=xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 3, alpha = 10, n_estimators = 400), n_estimators=7, learning_rate=0.05, loss='square', random_state=None)

In [0]:
Ada_fit = Ada_reg.fit(x_poly,y_train)

Prediction

In [0]:
Ada_pred = Ada_fit.predict(x_test)

In [0]:
test_df = pd.DataFrame({'Actual': y_test, 'Predicted': Ada_pred})
test_df.head()

,Actual,Predicted
0,8.250000,13.796331
1,22.366667,30.246632
2,68.133333,54.573921
3,32.033333,23.407330
4,10.466667,8.169461


Mean Squared Error

In [0]:
metrics.mean_squared_error(y_test, test_df['Predicted'])

147.96962407453714

Accuracy

In [0]:
print(r2_score(y_test,test_df['Predicted']))

0.4519365741343727


**GradientBoost regresser**

In [ ]:
gsc = GridSearchCV(
        estimator=GradientBoostingRegressor(),
        param_grid={
            'learning_rate': (0.1,0.07,0.08),
            'n_estimators': (100,200,300),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,n_jobs=-1)
    
grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_

In [0]:
Grad_reg = GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=4, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort='auto', validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)

In [0]:
Grad_fit=Grad_reg.fit(x_poly,y_train)

Prediction

In [0]:
Grad_pred = Grad_fit.predict(x_test)

In [0]:
test_df = pd.DataFrame({'Actual': y_test, 'Predicted': Grad_pred})
test_df.head()

,Actual,Predicted
0,8.250000,15.253814
1,22.366667,29.904732
2,68.133333,49.884703
3,32.033333,24.369487
4,10.466667,8.631892


Mean Squared Error

In [0]:
metrics.mean_squared_error(y_test, test_df['Predicted'])

149.7004073445024

Accuracy

In [0]:
print(r2_score(y_test,test_df['Predicted']))

0.4455259407743105


**XGBoost Regresser**

In [ ]:
gsc = GridSearchCV(
        estimator=xgb.XGBRegressor(),
        param_grid={
            'learning_rate': (0.1,0.07,0.08),
            'n_estimators': (400,500),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,n_jobs=-1)
    
grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_

In [0]:
Xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 3, alpha = 10, n_estimators = 400,booster='gbtree') 

In [0]:
Xg_fit=Xg_reg.fit(x_poly,y_train)

Prediction

In [0]:
Xg_pred = Xg_fit.predict(x_test)

In [0]:
test_df = pd.DataFrame({'Actual': y_test, 'Predicted': Xg_pred})
test_df.head()

,Actual,Predicted
0,8.250000,14.290668
1,22.366667,28.793074
2,68.133333,55.114517
3,32.033333,23.465504
4,10.466667,7.767723


Mean squared Error

In [0]:
metrics.mean_squared_error(y_test, test_df['Predicted'])

148.88742919603348

Accuracy


In [0]:
print(r2_score(y_test,test_df['Predicted']))

0.4485371235896385


**RandomForest Regresser**

In [ ]:
gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(5,10),
            'n_estimators': (400,500),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,n_jobs=-1)
    
grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_

In [0]:
Rf_reg=RandomForestRegressor(max_depth=10, n_estimators=500,random_state=False, verbose=False)

In [0]:
Rf_fit=Rf_reg.fit(x_poly,y_train)

Prediction

In [0]:
Rf_pred = Rf_fit.predict(x_test)

In [56]:
test_df = pd.DataFrame({'Actual': y_test, 'Predicted': Rf_pred})
test_df.head()

,Actual,Predicted
0,8.250000,13.821218
1,22.366667,30.561187
2,68.133333,48.113558
3,32.033333,31.152737
4,10.466667,8.866749


Mean Squared Error


In [57]:
metrics.mean_squared_error(y_test, test_df['Predicted'])

153.41261965268325

Accuracy

In [58]:
print(r2_score(y_test,test_df['Predicted']))

0.43177630933551414


**Avarage of AdaBoost, GradientBoost And XGBoost**


In [0]:
test_df = pd.DataFrame({'Actual': y_test, 'Predicted': (Ada_pred+Grad_pred+Xg_pred)/3})
test_df.head()


,Actual,Predicted
0,8.250000,14.466994
1,22.366667,29.797783
2,68.133333,51.732448
3,32.033333,26.287468
4,10.466667,8.086807


Mean squared Error

In [0]:
metrics.mean_squared_error(y_test, test_df['Predicted'])

147.8486510057253

Accuracy

In [0]:
print(r2_score(y_test,test_df['Predicted']))

0.45238464524994904


# **Testing Dataset**

In [0]:
col_names=['Order_No', 'User_Id', 'Vehicle_Type', 'Platform_Type',
       'Type', 'Placement_Day_of_Month',
       'Placement_Weekday', 'Placement_Time',
       'Confirmation_Day_of_Month', 'Confirmation_Weekday',
       'Confirmation_Time', 'Arrival_at_Pickup_Day_of_Month',
       'Arrival_at_Pickup_Weekday', 'Arrival_at_Pickup_Time',
       'Pickup_Day_of_Month', 'Pickup_Weekday', 'Pickup_Time',
        'Distance', 'Temperature',
       'Precipitation', 'Pickup_Lat', 'Pickup_Long',
       'Destination_Lat', 'Destination_Long', 'Rider_Id',]

In [0]:
test=pd.read_csv('Test.csv',names=col_names,skiprows=1)
test.head()

,Order_No,User_Id,Vehicle_Type,Platform_Type,Type,Placement_Day_of_Month,Placement_Weekday,Placement_Time,Confirmation_Day_of_Month,Confirmation_Weekday,Confirmation_Time,Arrival_at_Pickup_Day_of_Month,Arrival_at_Pickup_Weekday,Arrival_at_Pickup_Time,Pickup_Day_of_Month,Pickup_Weekday,Pickup_Time,Distance,Temperature,Precipitation,Pickup_Lat,Pickup_Long,Destination_Lat,Destination_Long,Rider_Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,4:44:29 PM,27,3,4:53:04 PM,27,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,12:59:17 PM,17,5,1:20:27 PM,17,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,11:25:05 AM,27,4,11:33:20 AM,27,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,1:53:27 PM,17,1,2:02:41 PM,17,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,11:34:45 AM,11,2,11:47:19 AM,11,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


Integrating Riders information

In [0]:
test = pd.merge(test,
                 riders[['Rider_Id', 'Average_Rating','Rider_exp']],
                 on='Rider_Id')

Generating time attributes

In [0]:
def hfmt(x):
  return x.strftime('%H:%M:%S')
test['Placement_Time']=pd.to_datetime(test['Placement_Time'])
test['Placement_Time']=test['Placement_Time'].apply(hfmt)
test['Placement_Time']=pd.to_timedelta(test['Placement_Time'])

test['Confirmation_Time']=pd.to_datetime(test['Confirmation_Time'])
test['Confirmation_Time']=test['Confirmation_Time'].apply(hfmt)
test['Confirmation_Time']=pd.to_timedelta(test['Confirmation_Time'])

test['Arrival_at_Pickup_Time']=pd.to_datetime(test['Arrival_at_Pickup_Time'])
test['Arrival_at_Pickup_Time']=test['Arrival_at_Pickup_Time'].apply(hfmt)
test['Arrival_at_Pickup_Time']=pd.to_timedelta(test['Arrival_at_Pickup_Time'])

test['Pickup_Time']=pd.to_datetime(test['Pickup_Time'])
test['Pickup_Time']=test['Pickup_Time'].apply(hfmt)
test['Pickup_Time']=pd.to_timedelta(test['Pickup_Time'])


In [0]:
test['conf_no']=test['Confirmation_Time']-test['Placement_Time']
conf_temp=pd.DatetimeIndex(test['conf_no'])
conf_temp=(conf_temp.hour*60+conf_temp.minute+(conf_temp.second/60))
test['confirmation_time_min']=conf_temp

test['Arrival_no']=test['Arrival_at_Pickup_Time']-test['Confirmation_Time']
conf_temp=pd.DatetimeIndex(test['Arrival_no'])
conf_temp=(conf_temp.hour*60+conf_temp.minute+(conf_temp.second/60))
test['Arrival_time_min']=conf_temp

test['Pickup_no']=test['Pickup_Time']-test['Arrival_at_Pickup_Time']
conf_temp=pd.DatetimeIndex(test['Pickup_no'])
conf_temp=(conf_temp.hour*60+conf_temp.minute+(conf_temp.second/60))
test['Pickup_time_min']=conf_temp


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning:

Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version



One hot encoding of platform types(1,2,3,4) and type(business,personal)

In [0]:
int_data=LabelEncoder().fit_transform(test['Platform_Type'])
one_hot_data=OneHotEncoder().fit_transform(int_data.reshape(-1,1)).toarray()
test['1']=one_hot_data[:,0]
test['2']=one_hot_data[:,1]
test['3']=one_hot_data[:,2]
test['4']=one_hot_data[:,3]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.



In [0]:
int_data=LabelEncoder().fit_transform(test['Type'])
one_hot_data=OneHotEncoder().fit_transform(int_data.reshape(-1,1)).toarray()
test['Business']=one_hot_data[:,0]
test['Personal']=one_hot_data[:,1]

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.



Handling Null values for temperature and precipitation

In [0]:
med_temp=test.Temperature.median()
test['Temperature'].fillna(med_temp,inplace=True)

In [0]:
test['Precipitation'].fillna(0,inplace=True)
test['Precipitation'].isnull().sum()

0

Inserting Rider average speeds

In [0]:
test.replace(xx.index,xx.values,inplace=True)

Inserting mean speed for those riders not present in the training Dataset

In [0]:
ttt=test.loc[test.Rider_Id.str.contains('^Rider_Id').notnull()]
yy=ttt['Rider_Id']
yy
test.replace(yy.values,2.2254,inplace=True)

Predicting for the testing dataset

In [0]:
test_data=test[['Personal','Business','Pickup_time_min','Distance','Rider_Id','new_attr','Destination_Lat','Destination_Long','Pickup_Lat','Pickup_Long']].values

In [0]:
polynomial_features= PolynomialFeatures(degree=2)
x_testing_poly=polynomial_features.fit_transform(test_data)

In [0]:
y_pred = Grad_fit.predict(x_testing_poly)
y_pred1 = Xg_fit.predict(x_testing_poly)
y_pred2 = Ada_fit.predict(x_testing_poly)
test['time_in_sec']=((y_pred+y_pred1+y_pred2)/3)*60

In [0]:
sub_file=test[['Order_No','time_in_sec']]
sub_file.head()

In [0]:
sub_file.columns=['Order_No','Time from Pickup to Arrival']

In [0]:
sub_file['Time from Pickup to Arrival']=sub_file['Time from Pickup to Arrival'].astype(int)
sub_file.head()

In [0]:
sub_file.to_csv('xgav1.csv',index=False)